In [5]:
from joblib import Parallel, delayed
from math import sqrt
from pathlib import Path
from lxml import etree
from tqdm import tqdm
import pickle
import dateparser

from bs4 import BeautifulSoup

import pandas as pd
import swifter

import pandas as pd
import sqlite3
from cleantext import clean

from pathlib import Path
import numpy as np
import swifter
from somajo import Tokenizer, SentenceSplitter
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed

from german_lemmatizer import lemmatize

num_cores = multiprocessing.cpu_count()

import numpy as np

In [ ]:
df = pd.read_pickle('/mnt/data2/ptf/cleaned.pkl')

In [ ]:
df = df.drop(columns=['cleaned'])

In [ ]:
slem_txt = list(lemmatize(df['text'].values[0:], n_jobs=3, chunk_size=10000))

In [ ]:
pickle.dump( slem_txt, open( "lemma.pkl", "wb" ) )

In [ ]:
slem_txt = pickle.load( open( "lemma.pkl", "rb" ) )

In [ ]:
def get_sents(texts):
    tokenizer = Tokenizer(split_camel_case=True, token_classes=False, extra_info=False)
    sentence_splitter = SentenceSplitter(is_tuple=False)
    
    results = []
    for text in texts:
#         text = clean(text, lang='de', lower=False)
        tokens = tokenizer.tokenize_paragraph(text)
        sentences = sentence_splitter.split(tokens)
        cleaned = [clean(' '.join(s), no_urls=True, no_digits=True, no_punct=True, no_line_breaks=True, lang='de') for s in sentences]
        results.append(cleaned)
    return results

In [ ]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [ ]:
results = Parallel(n_jobs=4)(delayed(get_sents)(row) for row in tqdm(list(chunks(slem_txt, 1000))))

In [6]:
results = pickle.load( open( "results_lemma.pkl", "rb" ) )

In [7]:
with open('/mnt/data2/ptf/zo2.txt', 'w') as outfile:
    for r in results:
        for d in r:
            outfile.write('\n'.join([ x for x in d]) + '\n')

In [ ]:
pickle.dump( results, open( "results_lemma.pkl", "wb" ) )

In [ ]:
len(results)

In [ ]:
len(slem_txt)

In [ ]:
# fucked up, need to either remove new lines or escape them for 

In [ ]:
for r in results[:100]:
    for d in r:
        print("\n".join(d))